In [1]:
from datetime import datetime
import math
import os

import pandas as pd
from zipfile import ZipFile

CITY = 'chicago'
print('city', CITY)


RIDES_COUNT_THRESHOLD = 100


def get_filepath(city):
    return '../data/' + city + '-bike/'


def transform_date(date):
    try:
        dt = datetime.strptime(date.split(' ')[0], '%m/%d/%Y')
    except ValueError:
        # this dataset is so frustrating lol
        dt = datetime.strptime(date.split(' ')[0], '%Y-%m-%d')
        
    return dt.strftime('%Y-%m-%d')

def open_zipfile(zipfilename):
    # Because someone dropped some gnarly mac osx files into their zips
    zipfile = ZipFile(zipfilename)
    filenames = [f.filename for f in zipfile.infolist()]
    # Return the first file that can be opened  - not all of them have .csv suffix
    for filename in filenames:
        try:
            df = pd.read_csv(zipfile.open(filename))
            return df
        except:
            print('failed to open filename from zip', zipfilename, ': ', filename)
            pass
    raise Exception('unable to read a csv from zipfile %s' % zipfilename)

city chicago


In [2]:
ID = 'id'
NAME = 'name'
LAT = 'lat'
LON = 'lon'
FIRST = 'first'
LAST = 'last'
RIDES = 'rides'

In [3]:
def preprocess_stations_df(df):  
    df.columns = df.columns.str.replace('01 - Rental Details Bike ID', 'bikeid')  
    df.columns = df.columns.str.replace('03 - Rental Start Station ID', 'from_station_id')
    df.columns = df.columns.str.replace('03 - Rental Start Station Name', 'from_station_name')
    df.columns = df.columns.str.replace('02 - Rental End Station ID', 'to_station_id')
    df.columns = df.columns.str.replace('02 - Rental End Station Name', 'to_station_name')
    df.columns = df.columns.str.replace('01 - Rental Details Local Start Time', 'start_time')
    df.columns = df.columns.str.replace('01 - Rental Details Local End Time', 'end_time')
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace('[\ ]', '')
    # transform the dates
    df[starttime] = df[starttime].apply(transform_date)
    if CITY == "chicago":
        df = preprocess_boston_stations_df(df)
    return df

In [4]:
bike_stations_locations_filenames = [
    "Divvy_Stations_2013.csv",
    "Divvy_Stations_2014-Q3Q4.csv",
    #"Divvy_Stations_2014-Q1Q2.xlsx",
    "Divvy_Stations_2015.csv",
    "Divvy_Stations_2016_Q1Q2.csv",
    "Divvy_Stations_2016_Q3.csv",
    "Divvy_Stations_2016_Q4.csv",
    "Divvy_Stations_2017_Q1Q2.csv",
    "Divvy_Stations_2017_Q3Q4.csv",
]
stations_filename_path={
    "Divvy_Stations_2013.csv":'Divvy_Stations_Trips_2013/Divvy_Stations_2013.csv',
    "Divvy_Stations_2014-Q1Q2.xlsx":'Divvy_Stations_Trips_2014_Q1Q2/Divvy_Stations_2014-Q1Q2.xlsx',
    "Divvy_Stations_2014-Q3Q4.csv":'Divvy_Stations_Trips_2014_Q3Q4/Divvy_Stations_2014-Q3Q4.csv',
    "Divvy_Stations_2015.csv":'Divvy_Trips_2015-Q1Q2/Divvy_Stations_2015.csv' ,
    "Divvy_Stations_2016_Q1Q2.csv":'Divvy_Trips_2016_Q1Q2/Divvy_Stations_2016_Q1Q2.csv',
    "Divvy_Stations_2016_Q3.csv":'Divvy_Trips_2016_Q3Q4/Divvy_Stations_2016_Q3.csv',
    "Divvy_Stations_2016_Q4.csv":'Divvy_Trips_2016_Q3Q4/Divvy_Stations_2016_Q4.csv',
    "Divvy_Stations_2017_Q1Q2.csv":'Divvy_Trips_2017_Q1Q2/Divvy_Stations_2017_Q1Q2.csv',
    "Divvy_Stations_2017_Q3Q4.csv":'Divvy_Trips_2017_Q3Q4/Divvy_Stations_2017_Q3Q4.csv' }

stations_filename_year = {
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2013.csv']:2013,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2014-Q3Q4.csv']:2014,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2014-Q1Q2.xlsx']:2014,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2015.csv']:2015,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2016_Q3.csv']:2016,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2016_Q4.csv']:2016,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2016_Q1Q2.csv']:2016,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2017_Q1Q2.csv']:2017,
    get_filepath(CITY) + stations_filename_path['Divvy_Stations_2017_Q3Q4.csv']:2017,
}

def get_bike_stations_locations_df():
    df = pd.DataFrame()
    filenames = [get_filepath(CITY) + stations_filename_path[fname] for fname in bike_stations_locations_filenames]
    for filename in filenames:
        new_df = pd.read_csv(filename)
        bike_stations_locations_column_names = {
            "latitude": 'lat',
            "longitude": 'lon',
        }
        new_df.rename(columns=bike_stations_locations_column_names, inplace=True)
        new_df.columns = new_df.columns.str.replace('online date', 'online_date')
        new_df['first']=stations_filename_year[filename]        
        df = new_df if df.empty else df.append(new_df)
    df.drop_duplicates(subset=['name'], inplace=True)
    return df
stations_df = None
if CITY == "chicago":
    stations_df = get_bike_stations_locations_df()
stations_df['last']=2019
stations_df = stations_df.drop(['city', 'Unnamed: 7','dateCreated','dpcapacity','landmark'], axis=1)
del stations_df['online_date']
#del bike_stations_locations_df['city']
#del bike_stations_locations_df['Unnamed: 7']
#del bike_stations_locations_df['dateCreated']
#del bike_stations_locations_df['dpcapacity']
#del bike_stations_locations_df['landmark']

def preprocess_boston_stations_df(df):
    if start_station_latitude in df.columns:
        return df
    # Otherwise this is one of the datasets that is lacking lat, lon info.
    # Add the lat,lon info
    return hubway_stations_locations_df.merge(df, on=start_station_id)


print('bikestations shape', stations_df.shape)
stations_df.head()

bikestations shape (661, 6)


/Users/xuwei/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,first,id,lat,lon,name,last
0,2013,5,41.873958,-87.627739,State St & Harrison St,2019
1,2013,13,41.932500,-87.652681,Wilton Ave & Diversey Pkwy,2019
2,2013,14,41.858086,-87.651073,Morgan St & 18th St,2019
3,2013,15,41.858181,-87.656487,Racine Ave & 18th St,2019
4,2013,16,41.910329,-87.672516,Wood St & North Ave,2019


In [5]:
bad_mappings = [] #new name and original id
bad_stations_names_to_ids={} #original name and original id
old_stations_names_to_last_rides={}
station_ids_to_lat={}
station_ids_to_lon={}
station_names_to_lat={}
station_names_to_lon={}
station_ids_to_names_map = {}
station_ids_to_new_names_map = {}
renewed_name_to_org_name_map = {}
org_name_to_renewed_name_map = {}
station_names_to_ids_map = {}

bad_station_idx=0

org_station_names=[]

for index, row in stations_df.iterrows():
    station_id = row['id']
    station_name = row['name']
    station_last_ride=row['first']
    station_lat=row['lat']
    station_lon=row['lon']
    if station_name not in org_station_names:
        org_station_names.append(station_name)
    if station_id in station_ids_to_names_map:
        if station_name != station_ids_to_names_map[station_id]:
            if station_ids_to_lat[station_id]!=station_lat or station_ids_to_lon[station_id]!=station_lon:
                print('we found a bad station by id', station_id, station_name, index, station_last_ride)
                bad_mappings.append((station_id, station_name))
                bad_stations_names_to_ids[station_name]=station_id
                old_stations_names_to_last_rides[station_ids_to_names_map[station_id]]=station_last_ride
                bad_station_idx+=1
            else:
                station_ids_to_new_names_map[station_id]=station_name
                renewed_name_to_org_name_map[station_name]=station_ids_to_names_map[station_id]
                org_name_to_renewed_name_map[station_ids_to_names_map[station_id]] = station_name
                stations_df.at[index,'name']=station_ids_to_names_map[station_id]
            continue
    if station_name in station_names_to_ids_map:
        if station_id != station_names_to_ids_map[station_name]:
            print('we found a bad mapping by name', station_id, station_name,station_last_ride)
            bad_mappings.append((station_id, station_name))
            continue

    station_ids_to_names_map[station_id] = station_name
    station_ids_to_lat[station_id]=station_lat
    station_ids_to_lon[station_id]=station_lon
    station_names_to_ids_map[station_name] = station_id
    
stations_df.drop_duplicates(subset=['name'], inplace=True)

for index, row in stations_df.iterrows():
    station_name = row['name']
    if station_name in org_name_to_renewed_name_map:
        stations_df.at[index,'name'] = org_name_to_renewed_name_map[station_name]

print(len(org_station_names))
print (len(bad_mappings),len(station_ids_to_lat),len(station_ids_to_names_map),len(station_names_to_ids_map))


print('bikestations shape', stations_df.shape)

stations_df.head(10)

we found a bad station by id 15 Racine Ave & 19th St 3 2014
we found a bad station by id 58 Marshfield Ave & Cortland St 41 2014
we found a bad station by id 84 Union Ave & Grand Ave 59 2014
we found a bad station by id 94 Clark St & Armitage Ave 68 2014
we found a bad station by id 163 Damen Ave & Clybourn Ave 121 2014
we found a bad station by id 165 Clark St & Grace St 123 2014
we found a bad station by id 194 Wabash Ave & Wacker Pl 147 2014
we found a bad station by id 196 Cityfront Plaza Dr & Pioneer Ct 149 2014
we found a bad station by id 212 Wells St & Hubbard St 165 2014
we found a bad station by id 232 Pine Grove Ave & Waveland Ave 184 2014
we found a bad station by id 311 Leavitt St & Lawrence Ave 259 2014
we found a bad station by id 344 Ravenswood Ave & Lawrence Ave 292 2014
we found a bad station by id 109 900 W Harrison St 95 2015
we found a bad station by id 110 Dearborn St & Erie St 96 2015
we found a bad station by id 273 Michigan Ave & 16th St 250 2015
we found a bad

,first,id,lat,lon,name,last
0,2013,5,41.873958,-87.627739,State St & Harrison St,2019
1,2013,13,41.932500,-87.652681,Wilton Ave & Diversey Pkwy,2019
2,2013,14,41.858086,-87.651073,Morgan St & 18th St,2019
3,2013,15,41.858181,-87.656487,Racine Ave & 18th St,2019
4,2013,16,41.910329,-87.672516,Wood St & North Ave,2019
5,2013,17,41.903320,-87.672730,Wood St & Division St,2019
6,2013,19,41.869417,-87.660996,Loomis St & Taylor St (*),2019
7,2013,20,41.909592,-87.653497,Sheffield Ave & Kingsbury St,2019
8,2013,21,41.877726,-87.654787,Aberdeen St & Jackson Blvd,2019
9,2013,22,41.869482,-87.655486,May St & Taylor St,2019


In [6]:
for index, row in stations_df.iterrows():
    station_id = row['id']
    station_name = row['name']
    station_lat=row['lat']
    station_lon=row['lon']
    station_last_ride=row['first']
    station_names_to_lat[station_name]=station_lat
    station_names_to_lon[station_name]=station_lon
    if station_name in old_stations_names_to_last_rides:
        stations_df.at[index,'last']=old_stations_names_to_last_rides[station_name]

print(stations_df.shape,len(station_names_to_lon),len(station_names_to_lat))
stations_df.head(30)

(631, 6) 628 628


,first,id,lat,lon,name,last
0,2013,5,41.873958,-87.627739,State St & Harrison St,2016
1,2013,13,41.932500,-87.652681,Wilton Ave & Diversey Pkwy,2019
2,2013,14,41.858086,-87.651073,Morgan St & 18th St,2019
3,2013,15,41.858181,-87.656487,Racine Ave & 18th St,2014
4,2013,16,41.910329,-87.672516,Wood St & North Ave,2016
5,2013,17,41.903320,-87.672730,Wood St & Division St,2017
6,2013,19,41.869417,-87.660996,Loomis St & Taylor St (*),2019
7,2013,20,41.909592,-87.653497,Sheffield Ave & Kingsbury St,2019
8,2013,21,41.877726,-87.654787,Aberdeen St & Jackson Blvd,2019
9,2013,22,41.869482,-87.655486,May St & Taylor St,2019


In [7]:
bike_trips_filenames = [
    "Divvy_Trips_2013.csv",
    "Divvy_Trips_2014_Q1Q2.csv",
    "Divvy_Trips_2014-Q3-07.csv",
    "Divvy_Trips_2014-Q3-0809.csv",
    "Divvy_Trips_2014-Q4.csv",
    "Divvy_Trips_2015-Q1.csv",
    "Divvy_Trips_2015-Q2.csv",
    "Divvy_Trips_2015_07.csv",
    "Divvy_Trips_2015_08.csv",
    "Divvy_Trips_2015_09.csv",
    "Divvy_Trips_2015_Q4.csv",
    "Divvy_Trips_2016_Q1.csv",
    "Divvy_Trips_2016_04.csv",
    "Divvy_Trips_2016_05.csv",
    "Divvy_Trips_2016_06.csv",
    "Divvy_Trips_2016_Q3.csv",
    "Divvy_Trips_2016_Q4.csv",
    "Divvy_Trips_2017_Q1.csv",
    "Divvy_Trips_2017_Q2.csv",
    "Divvy_Trips_2017_Q3.csv",
    "Divvy_Trips_2017_Q4.csv",
    "Divvy_Trips_2018_Q1.csv",
    "Divvy_Trips_2018_Q2.csv",
    "Divvy_Trips_2018_Q3.csv",
    "Divvy_Trips_2018_Q4.csv",
    "Divvy_Trips_2019_Q3.csv",
]

trips_filename_path={
    "Divvy_Trips_2013.csv":'Divvy_Stations_Trips_2013/Divvy_Trips_2013.csv',
    "Divvy_Trips_2014_Q1Q2.csv":'Divvy_Stations_Trips_2014_Q1Q2/Divvy_Trips_2014_Q1Q2.csv',
    "Divvy_Trips_2014-Q3-07.csv":'Divvy_Stations_Trips_2014_Q3Q4/Divvy_Trips_2014-Q3-07.csv',
    "Divvy_Trips_2014-Q3-0809.csv":'Divvy_Stations_Trips_2014_Q3Q4/Divvy_Trips_2014-Q3-0809.csv',
    "Divvy_Trips_2014-Q4.csv":'Divvy_Stations_Trips_2014_Q3Q4/Divvy_Trips_2014-Q4.csv',
    "Divvy_Trips_2015-Q1.csv":'Divvy_Trips_2015-Q1Q2/Divvy_Trips_2015-Q1.csv',
    "Divvy_Trips_2015-Q2.csv":'Divvy_Trips_2015-Q1Q2/Divvy_Trips_2015-Q2.csv',
    "Divvy_Trips_2015_07.csv":'Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_07.csv',
    "Divvy_Trips_2015_08.csv":'Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_08.csv',
    "Divvy_Trips_2015_09.csv":'Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_09.csv',
    "Divvy_Trips_2015_Q4.csv":'Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_Q4.csv',
    "Divvy_Trips_2016_Q1.csv":'Divvy_Trips_2016_Q1Q2/Divvy_Trips_2016_Q1.csv',
    "Divvy_Trips_2016_04.csv":'Divvy_Trips_2016_Q1Q2/Divvy_Trips_2016_04.csv',
    "Divvy_Trips_2016_05.csv":'Divvy_Trips_2016_Q1Q2/Divvy_Trips_2016_05.csv',
    "Divvy_Trips_2016_06.csv":'Divvy_Trips_2016_Q1Q2/Divvy_Trips_2016_06.csv',
    "Divvy_Trips_2016_Q3.csv":'Divvy_Trips_2016_Q3Q4/Divvy_Trips_2016_Q3.csv',
    "Divvy_Trips_2016_Q4.csv":'Divvy_Trips_2016_Q3Q4/Divvy_Trips_2016_Q4.csv',
    "Divvy_Trips_2017_Q1.csv":'Divvy_Trips_2017_Q1Q2/Divvy_Trips_2017_Q1.csv',
    "Divvy_Trips_2017_Q2.csv":'Divvy_Trips_2017_Q1Q2/Divvy_Trips_2017_Q2.csv',
    "Divvy_Trips_2017_Q3.csv":'Divvy_Trips_2017_Q3Q4/Divvy_Trips_2017_Q3.csv',
    "Divvy_Trips_2017_Q4.csv":'Divvy_Trips_2017_Q3Q4/Divvy_Trips_2017_Q4.csv',
    "Divvy_Trips_2018_Q1.csv":"Divvy_Trips_2018_Q1.csv",
    "Divvy_Trips_2018_Q2.csv":"Divvy_Trips_2018_Q2.csv",
    "Divvy_Trips_2018_Q3.csv":"Divvy_Trips_2018_Q3.csv",
    "Divvy_Trips_2018_Q4.csv":"Divvy_Trips_2018_Q4.csv",
    "Divvy_Trips_2019_Q3.csv":"Divvy_Trips_2019_Q3.csv",
}

new_bike_trips_filenames=[
    "Divvy_Trips_2018_Q1.csv",
    "Divvy_Trips_2018_Q2.csv",
    "Divvy_Trips_2018_Q3.csv",
    "Divvy_Trips_2018_Q4.csv",
]

def get_bike_trips_df(filename):
    new_df = pd.read_csv(filename)
    new_df.columns = new_df.columns.str.replace('01 - Rental Details Bike ID', 'bikeid')  
    new_df.columns = new_df.columns.str.replace('03 - Rental Start Station ID', 'from_station_id')
    new_df.columns = new_df.columns.str.replace('03 - Rental Start Station Name', 'from_station_name')
    new_df.columns = new_df.columns.str.replace('02 - Rental End Station ID', 'to_station_id')
    new_df.columns = new_df.columns.str.replace('02 - Rental End Station Name', 'to_station_name')
    new_df.columns = new_df.columns.str.replace('01 - Rental Details Local Start Time', 'start_time')
    new_df.columns = new_df.columns.str.replace('01 - Rental Details Local End Time', 'end_time')       
    return new_df
#go to https://data.cityofchicago.org/Transportation/Divvy-Bicycle-Stations/bbyy-e7gq/data and download station data,export as csv
def get_station_loc():
    new_df = pd.read_csv(get_filepath(CITY)+'Divvy_Bicycle_Stations.csv')
    bike_stations_locations_column_names = {
            "Latitude": 'lat',
            "Longitude": 'lon',
            "ID":'id',
            "Station Name": 'name',
        }
    new_df.rename(columns=bike_stations_locations_column_names, inplace=True)
    return new_df
if CITY =='chicago':
    all_station_loc_df = get_station_loc()
new_names_to_lat={}
new_names_to_lon={}
for index, row in all_station_loc_df.iterrows():
    station_name = row['name']
    station_lat=row['lat']
    station_lon=row['lon']
    new_names_to_lat[station_name]=station_lat
    new_names_to_lon[station_name]=station_lon
print(len(new_names_to_lat))
all_station_loc_df.head(30)

611


,id,name,Address,Total Docks,Docks in Service,Status,lat,lon,Location
0,102,Stony Island Ave & 67th St,NaN,11,11,In Service,41.773458,-87.585340,"(41.77345849948, -87.5853397391)"
1,103,Clinton St & Polk St,NaN,15,15,In Service,41.871467,-87.640949,"(41.87146651779, -87.6409491327)"
2,11,Jeffery Blvd & 71st St,NaN,11,11,In Service,41.766638,-87.576450,"(41.76663823695, -87.5764501141)"
3,12,South Shore Dr & 71st St,NaN,15,15,In Service,41.766409,-87.565688,"(41.7664094567, -87.5656875719)"
4,132,Wentworth Ave & 24th St (Temp),NaN,15,15,In Service,41.850084,-87.632141,"(41.85008369347577, -87.6321405172348)"
5,133,Kingsbury St & Kinzie St,NaN,31,31,In Service,41.889177,-87.638506,"(41.88917683258, -87.6385057718)"
6,145,Mies van der Rohe Way & Chestnut St,NaN,19,19,In Service,41.898587,-87.621915,"(41.8985866514, -87.6219152258)"
7,156,Clark St & Wellington Ave,NaN,15,15,In Service,41.936497,-87.647539,"(41.9364968219, -87.6475386582)"
8,157,Lake Shore Dr & Wellington Ave,NaN,15,15,In Service,41.936688,-87.636829,"(41.9366884494997, -87.63682901859283)"
9,161,Rush St & Superior St,NaN,23,23,In Service,41.895765,-87.625908,"(41.89576474564, -87.6259080327)"


In [8]:
new_stations=[]
new_stations_to_id_map={}
new_stations_to_first={}

new_trips_filenames = [get_filepath(CITY) + fname for fname in new_bike_trips_filenames]
trip_df = None
for filename in new_trips_filenames:
    trips_df = get_bike_trips_df(filename)
    print('handling',filename)
    for index, row in trips_df.iterrows():
        station_name = row['from_station_name']
        station_id = row['from_station_id']
        if station_name not in org_station_names and station_name not in new_stations:
            new_stations.append(station_name)
            new_stations_to_id_map[station_name]=station_id
            new_stations_to_first[station_name]=2018
            
print(len(new_stations),new_stations)

handling ../data/chicago-bike/Divvy_Trips_2018_Q1.csv
handling ../data/chicago-bike/Divvy_Trips_2018_Q2.csv
handling ../data/chicago-bike/Divvy_Trips_2018_Q3.csv
handling ../data/chicago-bike/Divvy_Trips_2018_Q4.csv
52 ['Winthrop Ave & Lawrence Ave', 'Winchester (Ravenswood) Ave & Balmoral Ave', 'Halsted St & Clybourn Ave (*)', 'Clark St & Winnemac Ave (Temp)', 'Damen Ave & Thomas St (Augusta Blvd)', 'Burling St (Halsted) & Diversey Pkwy (Temp)', 'Wentworth Ave & Cermak Rd (Temp)', 'Financial Pl & Congress Pkwy (Temp)', 'Pulaski Rd & Eddy St (Temp)', 'Clark St & Drummond Pl', 'Ashland Ave & Lake St (Temp)', 'MLK Jr Dr & Pershing Rd', 'Wolcott (Ravenswood) Ave & Montrose Ave (*)', 'Central Park Ave & 24th St', 'DIVVY Map Frame B/C Station', 'BBB ~ Divvy Parts Testing', 'TS ~ DIVVY PARTS TESTING', 'LaSalle Dr & Huron St (*)', 'Karlov Ave & Madison St', 'Aberdeen St & Randolph St', 'DIVVY CASSETTE REPAIR MOBILE STATION', 'Lincoln Ave & Sunnyside Ave', 'Malcolm X College', 'Kildare Ave & M

In [9]:
trips2019_df = get_bike_trips_df(get_filepath(CITY)+"Divvy_Trips_2019_Q3.csv")
print ('handling Divvy_Trips_2019_Q3.csv')
for index, row in trips2019_df.iterrows():
    station_name = row['from_station_name']
    station_id = row['from_station_id']
    if station_name not in org_station_names and station_name not in new_stations:
        new_stations.append(station_name)
        new_stations_to_id_map[station_name]=station_id
        new_stations_to_first[station_name]=2019
print(len(new_stations))

handling Divvy_Trips_2019_Q3.csv
78


In [10]:
l=len(new_stations)
station_without_loc=[]
for i in range(l-1,-1,-1):
    station_name=new_stations[i]
    new=True
    if station_name not in new_names_to_lat:
        station_without_loc.append(station_name)
        del(new_stations[i])
        continue
    for station in station_names_to_lat:
        if station_names_to_lat[station]==new_names_to_lat[station_name] and station_names_to_lon[station]==new_names_to_lon[station_name]:
            new=False
            break       
    if not new:
        del(new_stations[i])
print(len(station_without_loc),len(new_stations))
new_stations.sort()
station_without_loc.sort()
print(new_stations)
print(station_without_loc)

15 61
['Aberdeen St & Randolph St', 'Archer (Damen) Ave & 37th St', 'Bernard St & Elston Ave', 'Burling St (Halsted) & Diversey Pkwy (Temp)', 'Carpenter St & 63rd St', 'Central Park Ave & 24th St', 'Central Park Ave & Bloomingdale Ave', 'Cherry Ave & Blackhawk St', 'Clark St & Drummond Pl', 'Clark St & Ida B Wells Dr', 'Clark St & Newport St', 'Clinton St & Jackson Blvd', 'Cornell Dr & Hayes Dr', 'Damen Ave & Thomas St (Augusta Blvd)', 'Damen Ave & Walnut (Lake) St (*)', 'Dodge Ave & Mulford St', 'Eggleston Ave & 69th St (*)', 'Elizabeth (May) St & Fulton St', 'Evanston Civic Center', 'Fairbanks St & Superior St', 'Financial Pl & Ida B Wells Dr', 'Franklin St & Adams St (Temp)', 'Franklin St & Illinois St', 'Halsted St & Clybourn Ave', 'Hoyne Ave & Balmoral Ave', 'Karlov Ave & Madison St', 'Kildare Ave & Montrose Ave', 'LaSalle Dr & Huron St', 'Lakefront Trail & Wilson Ave', 'Latrobe Ave & Chicago Ave', 'Leavitt St & Belmont Ave (*)', 'Leavitt St & Chicago Ave', 'Leavitt St & Division 

In [11]:
def normalized_names(name):
    normalized_name = name.lower()
    normalized_name = normalized_name.replace("(Temp)",  "").replace(" ", "").replace('(*)','')
    return normalized_name
normal_new_station=[]
normal_station_without_loc=[]
for name in new_stations:
    normal_new_station.append(normalized_names(name))
for name in station_without_loc:
    normal_station_without_loc.append(normalized_names(name))
renewed_new_names={}
for i in range(len(new_stations)-1,-1,-1):
    name = normal_new_station[i]
    if name in normal_station_without_loc:
        j=normal_station_without_loc.index(name)
        renewed_new_names[new_stations[i]]=station_without_loc[j]
        station_without_loc.remove(station_without_loc[j])
print(len(station_without_loc))

9


In [12]:
print(station_without_loc)

['Ashland Ave & Lake St (Temp)', 'BBB ~ Divvy Parts Testing', 'Clark St & Winnemac Ave (Temp)', 'DIVVY CASSETTE REPAIR MOBILE STATION', 'DIVVY Map Frame B/C Station', 'Financial Pl & Congress Pkwy (Temp)', 'Franklin St & Chicago Ave (Temp)', 'HUBBARD ST BIKE CHECKING (LBS-WH-TEST)', 'TS ~ DIVVY PARTS TESTING']


In [13]:
new_id_to_names_map={}
for station in new_stations_to_id_map:
    new_id_to_names_map[new_stations_to_id_map[station]]=station
for index, row in stations_df.iterrows():
    station_id = row['id']
    station_name = row['name']
    if station_id in new_id_to_names_map and row['last']==2019:
        stations_df.at[index,'last']=new_stations_to_first[new_id_to_names_map[station_id]] 
for stn in new_stations:
    if stn not in renewed_new_names:
        stations_df=stations_df.append({'id':new_stations_to_id_map[stn],
                                        'name':stn,
                                        'lat':new_names_to_lat[stn],
                                        'lon':new_names_to_lon[stn],
                                        'first':new_stations_to_first[stn],
                                        'last':2019},ignore_index=True)
    else:
        stations_df=stations_df.append({'id':new_stations_to_id_map[stn],
                                        'name':renewed_new_names[stn],
                                        'lat':new_names_to_lat[stn],
                                        'lon':new_names_to_lon[stn],
                                        'first':new_stations_to_first[stn],
                                        'last':2019},ignore_index=True) 
stations_df.at[165,'last']=2014
print(stations_df.shape)
stations_df.tail(50)

(692, 6)


,first,id,lat,lon,name,last
642,2019,638,41.878116,-87.639838,Clinton St & Jackson Blvd (*),2019
643,2018,653,41.780595,-87.584846,Cornell Dr & Hayes Dr,2019
644,2018,183,41.901315,-87.677409,Damen Ave & Thomas St (Augusta Blvd),2019
645,2018,656,41.885951,-87.677009,Damen Ave & Walnut (Lake) St (*),2019
646,2018,662,42.023463,-87.699595,Dodge Ave & Mulford St,2019
647,2018,650,41.768898,-87.635876,Eggleston Ave & 69th St (*),2019
648,2019,217,41.886616,-87.658040,Elizabeth (May) St & Fulton St,2019
649,2018,661,42.057044,-87.686554,Evanston Civic Center,2019
650,2019,635,41.895748,-87.620104,Fairbanks St & Superior St (*),2019
651,2019,89,41.875024,-87.633094,Financial Pl & Ida B Wells Dr,2019


In [14]:
# Save the data to CSV
save_to_csvfilename = get_filepath(CITY) + 'stations.csv'
stations_df.to_csv(save_to_csvfilename)
print('wrote data to ', save_to_csvfilename)

wrote data to  ../data/chicago-bike/stations.csv


In [15]:
import json

stations = []
for index, row in stations_df.iterrows():
    stations.append({
        ID: str(row[ID]),
        NAME: row[NAME],
        LAT: row[LAT],
        LON: row[LON],
        FIRST: row[FIRST],
        LAST: row[LAST],
    })

json = json.dumps(stations)

save_to_jsonfilename = get_filepath(CITY) + 'stations.json'
with open(save_to_jsonfilename, 'w') as f:
    f.write(json)
print("Data written to stations.json")

Data written to stations.json


In [16]:
stations_df.head()

,first,id,lat,lon,name,last
0,2013,5,41.873958,-87.627739,State St & Harrison St,2016
1,2013,13,41.932500,-87.652681,Wilton Ave & Diversey Pkwy,2019
2,2013,14,41.858086,-87.651073,Morgan St & 18th St,2019
3,2013,15,41.858181,-87.656487,Racine Ave & 18th St,2014
4,2013,16,41.910329,-87.672516,Wood St & North Ave,2016
